# FalkorDBVectorStore
<a href="https://docs.falkordb.com/" target="_blank">FalkorDB</a> is an open-source graph database with integrated support for vector similarity search

it supports:
- approximate nearest neighbor search
- Euclidean similarity & Cosine Similarity
- Hybrid search combining vector and keyword searches

This notebook shows how to use the FalkorDB vector index (`FalkorDB`)

See the <a href="https://docs.falkordb.com/" target="_blank">installation instruction</a>



## Setup

In [1]:
# Pip install necessary package
%pip install --upgrade  falkordb
%pip install --upgrade  tiktoken
%pip install --upgrade  langchain langchain_huggingface

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for falkordb: filename=falkordb-1.0.10-py3-none-any.whl size=32578 sha256=3bf2f4cb911777d9b18831105c78992a5259d7a51260619efa094fd162a57952
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\a8\75\f6\6f84885622d3e92d1ce64092b8c3f043c702385e1332ab9805
Successfully built falkordb
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/10.0 MB 7.5 MB/s eta 0:00:02
   -------

### Credentials
We want to use `HuggingFace` so we have to get the HuggingFace API Key

In [2]:
import getpass
import os

if "HUGGINGFACE_API_KEY" not in os.environ:
    os.environ["HUGGINGFACE_API_KEY"] = getpass.getpass("HuggingFace API Key:")

If you want to get automated tracing of your model calls you can also set your LangSmith API key by uncommenting below:

In [3]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

## Initialization

In [4]:
from langchain_community.vectorstores.falkordb_vector import FalkorDBVector
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

You can use FalkorDBVector locally with docker. See <a href="https://docs.falkordb.com/" target="_blank">installation instruction</a>

In [5]:
host = "localhost"
port = 6379

Or you can use FalkorDBVector with <a href="https://app.falkordb.cloud">FalkorDB Cloud</a>

In [6]:
# E.g
# host = "r-6jissuruar.instance-zwb082gpf.hc-v8noonp0c.europe-west1.gcp.f2e0a955bb84.cloud"
# port = 62471
# username = "falkordb" # SET ON FALKORDB CLOUD
# password = "password" # SET ON FALKORDB CLOUD

In [7]:
vector_store = FalkorDBVector(host=host, port=port, embedding=HuggingFaceEmbeddings())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\dell\Desktop\langchain\.venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dell\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ValueError: Could not connect to FalkorDB database.Please ensure that the host and port is correct

## Manage vector store

### Add items to vector store

In [ ]:
from langchain_core.documents import Document

document_1 = Document(page_content="foo", metadata={"source": "https://example.com"})

document_2 = Document(page_content="bar", metadata={"source": "https://example.com"})

document_3 = Document(page_content="baz", metadata={"source": "https://example.com"})

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents, ids=["1","2","3"])

['1', '2', '3']

### Update items in vector store

In [ ]:
updated_document = Document(
    page_content="qux", metadata={"source": "https://another-example.com"}
)

vector_store.update_documents(document_id="1", document=updated_document)

### Delete items from vector store

In [9]:
vector_store.delete(ids=["3"])

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent.

### Query directly

Performing a simple similarity search can be done as follows:

In [ ]:
results = vector_store.similarity_search(
    query="thud", k=1, filter={"source": "https://another-example.com"}
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* qux [{'text': 'qux', 'id': '1', 'source': 'https://another-example.com'}]


If you want to execute a similarity search and receive the corresponding scores you can run:

In [11]:
results = vector_store.similarity_search_with_score(query="bar")
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.000001] bar [{'text': 'bar', 'id': '2', 'source': 'https://example.com'}]


### Query by turning into retriever
You can also transform the vector store into a retriever for easier usage in your chains.

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("thud")

[Document(metadata={'text': 'qux', 'id': '1', 'source': 'https://another-example.com'}, page_content='qux')]

## Usage for retrieval-augmented generation
For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:
- <a href="https://python.langchain.com/v0.2/docs/tutorials/#working-with-external-knowledge" target="_blank">Tutorials: working with external knowledge</a>
- <a href="https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag" target="_blank">How-to: Question and answer with RAG</a>
- <a href="Retrieval conceptual docs" target="_blank">Retrieval conceptual docs</a>


## API reference
For detailed documentation of all FalkorDB__VectorStore features and configurations head to the API reference: https://api.python.langchain.com/en/latest/vectorstores/__module_name.vectorstores.__FalkorDB__VectorStore.html